In [ ]:
# Reads in photometry from different sources, normalizes them, and puts them
# onto a BJD time scale

# Created 2021 Dec. 28 by E.S.

In [1]:
import numpy as np
import pandas as pd
from astropy.time import Time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

%matplotlib qt

In [2]:
## USER INPUTS

photometry_source_file = "x_ari_asas3_vband_prepolished.txt"
epoch_writeout_file = "junk_jds_x_ari_asas3.csv" # generated as-we-go
bjd_file = "x_ari_asas3_bjd.csv" # to be read in after manual correction
writeout_polished_file = "x_ari_asas3_vband_polished.txt"

In [3]:
stem_super = "/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/"
stem_moore_macadam = stem_super + "polished_macadam_moore_photometry/"
stem_public_sources = stem_super + "all_photometry_program_stars/polished/"
bjd_sources = stem_super + "all_photometry_program_stars/bjds/"

In [6]:
# read in photometry

#df = pd.read_csv(stem_public_sources + photometry_source_file)
df = pd.read_csv(stem_public_sources + photometry_source_file, delim_whitespace=True)

In [7]:
# if there's a bunch of too-old stuff, remove it
#df = df.where(np.logical_and(df["JD"]>2456474-400,df["JD"]<2456474+400)).dropna(how="all").reset_index()
#df = df.where(df["JD"]<=2459238.567486600).dropna(how="all").reset_index()

In [8]:
df.keys()

Index(['HJD', 'MAG_3', 'MAG_0', 'MAG_1', 'MAG_2', 'MAG_4', 'MER_3', 'MER_0',
       'MER_1', 'MER_2', 'MER_4', 'GRADE', 'FRAME'],
      dtype='object')

In [9]:
df["HJD"] = np.add(2450000,df["HJD"])

In [10]:
# write out only JD, for conversion

#df["JD"] = np.add(df["obsmjd"],2400000.5)
#df["JD"].to_csv(epoch_writeout_file, header=False, index=False)
df["HJD"].to_csv(epoch_writeout_file, header=False, index=False)
print(epoch_writeout_file)

junk_jds_x_ari_asas3.csv


# at this point, do conversion step manually online

In [9]:
# FYI: location of Wichita (approx. center of U.S.)

'''
lat_wichita = 37.688889
lon_wichita = -97.336111
'''

'\nlat_wichita = 37.688889\nlon_wichita = -97.336111\n'

# read bjds back in and tack onto the dataframe

In [11]:
intermed_bjd = pd.read_csv(bjd_sources + bjd_file, names=["BJD"])
print(bjd_sources + bjd_file)

df["BJD"] = intermed_bjd.values

#df = df.join(intermed_bjd["BJD"])

/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/all_photometry_program_stars/bjds/x_ari_asas3_bjd.csv


In [12]:
len(df)

392

In [13]:
df

,HJD,MAG_3,MAG_0,MAG_1,MAG_2,MAG_4,MER_3,MER_0,MER_1,MER_2,MER_4,GRADE,FRAME,BJD
0,2.452454e+06,9.361,9.351,9.401,9.374,9.352,0.053,0.057,0.063,0.053,0.053,D,9760,2.452454e+06
1,2.452661e+06,9.571,9.522,9.581,9.570,9.568,0.026,0.050,0.041,0.025,0.027,A,35919,2.452661e+06
2,2.451889e+06,9.192,9.208,9.188,9.193,9.196,0.049,0.057,0.048,0.041,0.048,B,2940,2.451889e+06
3,2.451950e+06,9.574,9.598,9.567,9.575,9.581,0.038,0.063,0.045,0.033,0.033,A,9143,2.451950e+06
4,2.451956e+06,9.776,9.788,9.788,9.798,9.794,0.042,0.087,0.069,0.048,0.034,D,9981,2.451956e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,2.455107e+06,8.978,8.971,8.944,8.959,8.967,0.044,0.048,0.043,0.039,0.050,B,328834,2.455107e+06
388,2.455117e+06,9.482,9.469,9.458,9.467,9.460,0.040,0.049,0.042,0.037,0.046,B,330046,2.455117e+06
389,2.455124e+06,9.598,9.608,9.597,9.606,9.594,0.055,0.069,0.054,0.051,0.059,B,330763,2.455124e+06
390,2.455126e+06,9.279,9.325,9.343,9.292,9.244,0.052,0.100,0.069,0.048,0.057,D,331117,2.455126e+06


In [14]:
# in case of weird naming conventions

df["Magnitude"] = df["MAG_0"]
df["Uncertainty"] = df["MER_0"]


# look at the data, choose what we want

In [15]:
plt.clf()
plt.scatter(df["BJD"],
            df["Magnitude"])
plt.show()

In [11]:
plt.clf()
plt.scatter(df["BJD"],
            df["MAG_0"])
plt.scatter(df["BJD"],
            df["MAG_1"])
plt.scatter(df["BJD"],
            df["MAG_2"])
plt.scatter(df["BJD"],
            df["MAG_3"])
plt.scatter(df["BJD"],
            df["MAG_4"])
plt.show()

In [14]:
plt.clf()
plt.scatter(df["BJD"],
            df["mag_autocorr"])
plt.show()

In [13]:
plt.clf()
plt.scatter(df["JD"].where(df["Band"] == "V"),
            df["Magnitude"].where(df["Band"] == "V"), color="k")
plt.scatter(df["BJD"].where(df["Band"] == "V"),
            df["Magnitude"].where(df["Band"] == "V"), color="b")
plt.show()

In [18]:
plt.clf()
plt.scatter(df["BJD"].where(df["Observer Code"] == "SAH"),
            df["Magnitude"].where(df["Observer Code"] == "SAH"), color="k")
plt.show()

In [13]:
df.keys()

Index(['JD', 'Magnitude', 'Uncertainty', 'HQuncertainty', 'Band',
       'Observer Code', 'Comment Code(s)', 'Comp Star 1', 'Comp Star 2',
       'Charts', 'Comments', 'Transfomed', 'Airmass', 'Validation Flag',
       'Cmag', 'Kmag', 'HJD', 'Star Name', 'Observer Affiliation',
       'Measurement Method', 'Grouping Method', 'ADS Reference', 'Digitizer',
       'Credit', 'BJD'],
      dtype='object')

In [12]:
print(np.min(df["JD"]))
print(np.max(df["JD"]))

2439144.571
2459546.39706


In [11]:
len(df)

30529

In [15]:
df["Band"].value_counts()

KeyError: 'Band'

In [16]:
df["Observer Code"].value_counts()

KeyError: 'Observer Code'

In [16]:
# choose photometric subset

#df_subset_photo = df
#df_subset_photo = df.where(np.logical_and(df["BJD"] > 2452456,df["Band"] == "V"))
#df_subset_photo = df.where(np.logical_or(df["BJD"] < 2455109,df["BJD"] > 2455535))
#df_subset_photo = df.where(df["Band"] == "V")
df_subset_photo = df.where(df["MAG_0"] < 20)
#df_subset = df[["JD", "sdf"]]

In [17]:
# check to confirm
plt.clf()
plt.scatter(df_subset_photo["BJD"],df_subset_photo["Magnitude"])
#plt.scatter(df_subset_photo["BJD"],df_subset_photo["MAG_0"])
plt.show()

In [18]:
np.max(df_subset_photo["BJD"])

2455167.640255

In [19]:
df_subset_photo.keys()

Index(['HJD', 'MAG_3', 'MAG_0', 'MAG_1', 'MAG_2', 'MAG_4', 'MER_3', 'MER_0',
       'MER_1', 'MER_2', 'MER_4', 'GRADE', 'FRAME', 'BJD', 'Magnitude',
       'Uncertainty'],
      dtype='object')

# feature scale the photometry

### wait! scaling of error bars won't be the same, since an offset is being subtracted off

In [29]:
'''
scaler = MinMaxScaler() # min-max mapped to 0-to-1
df_subset_photo[['mag_scaled']] = scaler.fit_transform(df_subset_photo[['Magnitude']])
# df_subset_photo[['error_scaled']] = scaler.transform(df_subset_photo[['Uncertainty']])
'''

In [21]:
'''
plt.scatter(df_subset_photo["BJD"],df_subset_photo["mag_scaled"])
plt.show()
'''

# write out as 'polished' photometry

In [ ]:
# cols BJD, mag, error

In [20]:
# remove rows of all NaNs

df_subset_photo_dropna = df_subset_photo.dropna(axis=0, how='all')

In [21]:
# write out only the relevant cols

df_subset_photo_dropna.to_csv(stem_public_sources + writeout_polished_file, 
                              columns=["BJD","Magnitude","Uncertainty"], index=False)
print(stem_public_sources + writeout_polished_file)

/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/all_photometry_program_stars/polished/x_ari_asas3_vband_polished.txt


In [16]:
writeout_polished_file

't_sex_asas3_vband_polished.txt'